# 1-1-Assembly gap-filling 
Manuel Jara-Espejo$^{1}$

Aboobaker lab, Department of Biology, University of Oxford

## Contents of notebook
Introduction\
Run Samba on Parhyale reference genome *Phaw5.0*\
Run TGS-GapCloser \
Edit scaffold names

## Files
PacBio HiFi CCS reads: /drives/raid/AboobakerLab/manuel/data_phaw_analysis/asm_gap_filling/TGS-GapCloser_anlysis/m64164_211030_095407.hifi.subreads.fa \
Reference genome assembly: /drives/raid/AboobakerLab/data/genomes/phaw/parhyale_hawaiensis_15Feb20182_1IWOV.fa

## 1. Introduction

## 2. Run Samba on Parhyale reference genome *Phaw5.0*

In [1]:
%%bash

cd ./scripts
less 1_run_samba.sh

#!/bin/bash 

#Script used to Samba on Phaw5.0 (reference) assembly

/drives/raid/AboobakerLab/software/MaSuRCA-4.0.9/bin/samba.sh -r /drives/raid/AboobakerLab/data/genomes/phaw/parhyale_hawaiensis_15Feb20182_1IWOV.fa \
-q /drives/raid/AboobakerLab/manuel/data_phaw_analysis/asm_gap_filling/TGS-GapCloser_anlysis/m64164_211030_095407.hifi.subreads.fa \
-t 30 -d asm -m 5000 -o 1000 -a 

#Output assembly: parhyale_hawaiensis_15Feb20182_1IWOV.fa.scaffolds.fa

## 3. Run TGS-GapCloser

In [8]:
%%bash

cd ./scripts
less 2_run_tgsgapcloser_onSambaAsm.sh

#!/bin/bash

/drives/raid/AboobakerLab/software/TGS-GapCloser/TGS-GapCloser.sh \
--scaff  /drives/ssd1/manuel/phaw/2022_analysis/asm_SAMBA_analysis/parhyale_hawaiensis_15Feb20182_1IWOV.fa.scaffolds.fa \
--reads  /drives/raid/AboobakerLab/manuel/data_phaw_analysis/asm_gap_filling/TGS-GapCloser_anlysis/m64164_211030_095407.hifi.subreads.fa \
--output phaw_sambaAsm \
--minmap_arg '-x asm20' \
--ne \
--thread 20 \
--tgstype pb 

#Ouput assembly: phaw_sambaAsm.scaff_seqs
#The ouput file contained special characters in the scaffolds names, so I had to edit those.

## 4. Edit scaffold names

In [11]:
%%bash

cd ./scripts
less 3_edit_scaffold_names.rb

# After running Samba and TGSGpaCloser, the output: /drives/raid/AboobakerLab/manuel/data_phaw_analysis/asm_gap_filling/TGS-GapCloser_anlysis/sambaAsm_analysis/phaw_sambaAsm.scaff_seqs contained special characters.
# I used this ruby script to edit those names and generate the final assembly ouput Phaw5.1

##Before running the code below, convert gap-filled genome fasta file into tab-delimited file using: /drives/ssd1/manuel/FastaToTbl.sh phaw_sambaAsm.scaff_seqs > phaw_sambaAsm.scaff_seqs.tbl

count=1

File.open("phaw_sambaAsm.scaff_seqs.tbl","r").each do |line|
    if line.split("\t")[0].include?(":") 
        #puts #line.split("\t")[0] + "\t" "Scaffold_N" + count.to_s
        puts "Scaffold_N" + count.to_s  + "\t" + line.chomp.split("\t")[1]
        count += 1
    else
        puts line#.split("\t")[0]
    end
end

#Output: /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa, which

In [2]:
%%bash

cd ./scripts
less 4_IdentifyGaps.sh

#See gaps identified in Phaw5.0
head /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_fullAsm_stats/parhyale_hawaiensis_15Feb20182_1IWOV.fa.bed

echo ""

#See gaps identified in Phaw5.1
head /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/phaw_sambaAsm.scaff_seqs_editedScafNames.bed

#Commands used to identify gaps in Phaw5.0 and Phaw5.1 assemblies
##Command source: scaffoldgap2bed.py {genome} (https://github.com/lexnederbragt/sequencetools)

##Run command on Phaw5.0
nohup python3 /PATH/scaffoldgap2bed.py -i /drives/raid/AboobakerLab/manuel/data_phaw_analysis/phaw_reference/parhyale_hawaiensis_15Feb20182_1IWOV.fa -m 10 > parhyale_hawaiensis_15Feb20182_1IWOV.fa.bed &

##Run command on Phaw5.1
nohup python3 /PATH/scaffoldgap2bed.py -i /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa -m 10 > phaw_sambaAsm.scaff_seqs_editedScafNames.bed &

Scaffold_2;HRSCAF=42	1275	3240	gap
Scaffold_2;HRSCAF=42	3413	3513	gap
Scaffold_3;HRSCAF=66	7071	7908	gap
Scaffold_3;HRSCAF=66	8089	8189	gap
Scaffold_7;HRSCAF=96	994	2691	gap
Scaffold_7;HRSCAF=96	2823	2923	gap
Scaffold_8;HRSCAF=132	9503	9952	gap
Scaffold_8;HRSCAF=132	11420	14944	gap
Scaffold_8;HRSCAF=132	15854	16800	gap
Scaffold_8

In [34]:
%%bash

cd ./scripts
less 6_run_busco.sh

#!/bin/bash 

#Script to run busco analysis on phaw genome assembly

#Run busco on Phaw 5.0

if [ "$1" == "ref_assembly" ]; then

busco -i /drives/raid/AboobakerLab/manuel/data_phaw_analysis/phaw_reference/parhyale_hawaiensis_15Feb20182_1IWOV.fa -l arthropoda_odb10 --mode genome --cpu 16 -f --augustus -o ./busco_results_phaw5.0 

fi

#Run busco on Phaw 5.1
if [ "$1" == "gappfilled_assembly" ]; then

busco -i /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa -l arthropoda_odb10 --mode genome --cpu 16 -f --augustus -o ./busco_results_phaw5.1

fi
